In [ ]:
import random
import numpy as np
import time
import pandas as pd
import time


In [ ]:
#ONLY WORKS CORRECTLY UP TO 6
#This is sorting differently than lerio; he puts combinations that use less cards first

def find_ways(target, current_sum=0, start=1, path=[], combinations=[]):
    # Base case: if the current_sum equals the target, append the path to combinations
    if current_sum == target:
        combinations.append(path[:])  # Append a copy of the path to avoid modifying the original path
        return
    # Iterate through possible candidates to add to current_sum
    for i in range(start, target - current_sum + 1):
        # Update current_sum and path
        current_sum += i
        path.append(i)
        # Recursively call the function to find combinations for the remaining target
        find_ways(target, current_sum, i, path, combinations)
        # Backtrack: remove the last element from path and subtract from current_sum
        current_sum -= path.pop()

def perms(energy):
  target_number = energy
  combinations = []
  find_ways(target_number, combinations=combinations)
  combinations.reverse()
  max_values = [max(arr) for arr in combinations]
  combined_arrays = list(zip(combinations, max_values))
  # Sort the combined arrays based on the second column (maximum values) in decreasing order
  sorted_combined = sorted(combined_arrays, key=lambda x: x[1], reverse=True)
  # Remove the second column (maximum values)
  sorted_combined = [arr[0] for arr in sorted_combined]
  return sorted_combined


In [ ]:
#Monte Carlo problems for shuffling

def efficiency(n1, n2, n3, n4, n5, n6):
  #check parameters
  errorMessage = ""
  #check you have 12 cards
  if (n1 + n2 + n3 + n4 + n5 + n6) != 12:
    errorMessage += "All inputs must add to 12. "
  #check for negatives
  if n1 < 0 or n2 < 0 or n3 < 0 or n4 < 0 or n5 < 0 or n6 < 0:
    errorMessage += "All numbers must be 0 or greater. "
  #check for integers
  if not all(isinstance(x, int) for x in [n1, n2, n3, n4, n5, n6]):
    errorMessage += "All numbers must be integers. "
  if len(errorMessage) != 0:
    raise ValueError(errorMessage)
  #create deck
  deck = []
  costDist = [n1, n2, n3, n4, n5, n6]
  additon = 0
  for cost in costDist:
    additon += 1
    for i in range(cost):
      deck.append(additon)
  #randomly shuffle
  random.shuffle(deck)
  #deal
  hand = []
  board = []
  flag = False
  for i in range(3):
    hand.append(deck.pop())
  #for loop
  for i in range(1, 7):
    flag = False
    #draw card
    hand.append(deck.pop())
    #turn info for testing
    #print('Turn ', i)
    #print('Hand:', hand)
    #print('Played: ', board)
    #play cards
    for j in range(i, 0, -1):
      combos = perms(j)
      for k in range(len(combos)):
        cardPlays = combos[k]
        result = all(cardPlays.count(elem) <= hand.count(elem) for elem in set(cardPlays))
        if result:
          for l in range(len(cardPlays)):
            board.append(hand.pop(hand.index(cardPlays[l - 1])))
          flag = True
          break
      if flag:
        break
  return sum(board)


In [ ]:
#generate all profiles

def generate_lists(total, length, current_list=None, result=None):
    if current_list is None:
        current_list = []

    if result is None:
        result = []

    if total == 0 and len(current_list) == length:
        result.append(current_list[:])  # Append a copy of the current list
        return

    if total < 0 or len(current_list) >= length:
        return

    for i in range(0, 13):  # including 0 as a valid number
        generate_lists(total - i, length, current_list + [i], result)

    return result

combinations = generate_lists(12, 6)


In [ ]:
#test all profiles
start_time = time.time()

data = {'Profile': combinations, 'Efficiency': 0}

# Create the DataFrame
df = pd.DataFrame(data)

simulations = 1000
row = 0
for combo in combinations:
  NRGsum = 0
  for j in range(simulations):
    NRGsum += efficiency(combo[0], combo[1], combo[2], combo[3], combo[4], combo[5])
  df.loc[row, 'Efficiency'] = NRGsum
  if row % 500 == 0:
    print("First ", row, " rows completed.")
  row += 1
df['Efficiency'] = df['Efficiency'] / (simulations * 21)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


In [ ]:
df.to_excel('1000sims.xlsx', index=False)
